In [45]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Subset
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from collections import Counter
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter

# Additional Setup to use Tensorboard
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [20]:
# Helper function

def get_class_distribution(dataset):
    count_dict = {0:0, 1:0} # initialise dictionary
    
    for input, label in dataset:
        count_dict[label] += 1
            
    return count_dict

In [21]:
# transofrm images while loading

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),           # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

In [22]:
data_path = "data/random_3000"

In [23]:
# Load dataser
dataset = torchvision.datasets.ImageFolder(root=data_path, transform=transform)

In [24]:
# display class balance
dict(Counter(dataset.targets))

{0: 294, 1: 1705}

In [25]:
# Balanc the class by randomly samling 300 images from class 1

class1_limit = 300

idx_class1 = [i for i, label in enumerate(dataset.targets) if label == 1]
idx_class0 = [i for i, label in enumerate(dataset.targets) if label == 0]


np.random.shuffle(idx_class1)
idx_class1_limit = idx_class1[:300]
# print(len(idx_class1_limit))
# print(len(idx_class0))

idx_dataset_limited = np.concatenate((idx_class1_limit,idx_class0))
# print(len(idx_dataset_limited))

balanced_dataset = Subset(dataset, idx_dataset_limited)

In [26]:
get_class_distribution(balanced_dataset)

{0: 294, 1: 300}

In [27]:
train_size = int(0.8 * len(balanced_dataset))
test_size = len(balanced_dataset) - train_size
train_dataset, test_dataset = random_split(balanced_dataset, [train_size, test_size])

In [28]:
train_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [29]:
get_class_distribution(train_dataset)

{0: 200, 1: 203}

In [30]:
get_class_distribution(val_dataset)


{0: 32, 1: 40}

In [31]:
get_class_distribution(test_dataset)


{0: 62, 1: 57}

In [32]:
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)

In [33]:
resnet18 = models.resnet18(pretrained=True)

In [34]:
def train(train_loader, net, optimizer, criterion, device):
    """
    Trains network for one epoch in batches.

    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        criterion: Loss function (e.g. cross-entropy loss).
    """

    avg_loss = 0.
    correct = 0
    total = 0

    net.train()

    # iterate through batches
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero teh parameters of optimizer
        optimizer.zero_grad()

        # fwd + back + opti
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # keep track of loss and acc
        avg_loss += loss
        _, predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return avg_loss/len(train_loader), 100 * correct/total

def test(test_loader, net, criterion, device):
    avg_loss = 0.
    correct = 0
    total = 0

    net.eval()

    with torch.no_grad():
        for data in test_loader:

            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)


            outputs = net(inputs)
            loss = criterion(outputs, labels)

            avg_loss += loss
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return avg_loss/len(train_loader), 100 * correct/total


In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [41]:
writer = SummaryWriter()

num_classes = len(dataset.classes)

resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.fc.parameters(), lr=0.001)

num_epochs = 10
resnet18.to(device)

# Set the number of epochs to for training
epochs = 100

patience = 20

val_acc_best = 0
patience_cnt = 0

for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
    # Train on data
    train_loss, train_acc = train(train_loader, resnet18, optimizer, criterion, device)

    # Test on data
    val_loss, val_acc = test(val_loader, resnet18, criterion, device)

    # Write metrics to Tensorboard
    writer.add_scalars("Loss", {'Train': train_loss, 'Test':val_loss}, epoch)
    writer.add_scalars('Accuracy', {'Train': train_acc,'Test':val_acc} , epoch)


    if val_acc > val_acc_best:
      val_acc_best = val_acc
      patience_cnt = 0
      best_model_wts = resnet18.state_dict()

    else:
      patience_cnt += 1
      if patience_cnt == patience:
        break
    # print(f"Current loss {train_loss} at epoch {epoch}")


print('Finished Training')
writer.flush()
writer.close()



  0%|          | 0/100 [00:00<?, ?it/s]

 32%|███▏      | 32/100 [02:47<05:55,  5.23s/it]

Finished Training


In [38]:
# Open Tensorboard
%tensorboard --logdir runs/

# For local users only: uncomment the last line, run this cell once and wait for
# it to time out, run this cell a second time and you should see the board.
# %tensorboard --logdir runs/ --host localhost

Reusing TensorBoard on port 6006 (pid 6478), started 0:01:02 ago. (Use '!kill 6478' to kill it.)

In [42]:
# Final evaluation:

resnet18.load_state_dict(best_model_wts)

<All keys matched successfully>

In [44]:
torch.save(resnet18.state_dict(), "resnet18_model.pt")

In [46]:
true_labels = []
predicted_labels = []

resnet18.eval()


# Iterate through the test data
for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    
    # Forward pass
    with torch.no_grad():
        outputs = resnet18(inputs)
    
    _, predicted = torch.max(outputs, 1)
    
    # Append true labels and predicted labels
    true_labels.extend(labels.cpu().numpy())
    predicted_labels.extend(predicted.cpu().numpy())

# Calculate evaluation metrics
report = classification_report(true_labels, predicted_labels)

# Print the evaluation report
print(report)

              precision    recall  f1-score   support

           0       0.76      0.63      0.69        62
           1       0.66      0.79      0.72        57

    accuracy                           0.71       119
   macro avg       0.71      0.71      0.71       119
weighted avg       0.72      0.71      0.70       119

